In [1]:
import random
from config import DK_USERNAME,DK_PASSWORD

In [2]:
import os
import time
from selenium import webdriver
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : os.getcwd()+"/data"}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(chrome_options=chromeOptions)  # Optional argument, if not specified will search path.

In [30]:
driver.get("http://www.draftkings.com")

In [31]:
sign_in = driver.find_element_by_link_text('SIGN IN')
sign_in.click()

In [32]:
driver.find_element_by_name('username').send_keys(DK_USERNAME)

In [33]:
driver.find_element_by_name('username').send_keys("\t"+DK_PASSWORD)

In [34]:
driver.find_elements_by_xpath("//*[contains(text(), 'LOG IN')]")[0].click()

In [36]:
driver.get("https://www.draftkings.com/lobby#/NFL")

In [37]:
driver.find_elements_by_xpath("//*[contains(text(), 'View Live Contests')]")[0].click()

In [19]:
prize_pool=driver.find_elements_by_css_selector("div.ui-state-default.slick-header-column.grid-text-with-icon")[2]
prize_pool.click()
prize_pool.click()

In [20]:
from bs4 import BeautifulSoup as soup
import pandas as pd

In [21]:
table=soup(driver.find_element_by_id("lobby-grid").get_attribute("innerHTML"),"lxml")

In [195]:
old_contests=pd.read_csv(file_name)
full_contests=pd.concat([old_contests,live_contests]).drop_duplicates(subset=["link"])
full_contests

,Entries,Fee,Payout,Time,Unnamed: 0,link,name,time_pulled
0,15.1K/17.6K,$4,"$60,000",LIVE!,0.0,/contest/gamecenter/43299739,NFL $60K Front Four (Preseason),2017-08-11 22:49:35.450216
1,7058/7058,$0.25,"$1,500",LIVE!,1.0,/contest/gamecenter/43293788,NFL $1.5K Quarter Jukebox [Just $0.25!] (Prese...,2017-08-11 22:49:35.450216
2,4705/4705,$1,"$4,000",LIVE!,2.0,/contest/gamecenter/43293787,NFL $4K First Down (Preseason),2017-08-11 22:49:35.450216
3,2370/2823,$0.25,200 Tickets,LIVE!,3.0,/contest/gamecenter/43293832,NFL $2M Play-Action SUPERSatellite [200x] (Pre...,2017-08-11 22:49:35.450216
4,1882/1882,$0.25,20 Tickets,LIVE!,4.0,/contest/gamecenter/43293830,NFL $5.55M Fantasy Football Millionaire SUPERS...,2017-08-11 22:49:35.450216
5,1176/1176,$3,"$3,000",LIVE!,5.0,/contest/gamecenter/43299737,NFL $3K Play-Action (Preseason),2017-08-11 22:49:35.450216
6,976/1058,$1,300 Tickets,LIVE!,6.0,/contest/gamecenter/43293831,NFL $2M Play-Action SUPERSatellite [300x] (Pre...,2017-08-11 22:49:35.450216
7,882/882,$1,$750,LIVE!,7.0,/contest/gamecenter/43313515,NFL $750 First Down (Preseason),2017-08-11 22:49:35.450216
8,784/784,$3,"$2,000",LIVE!,8.0,/contest/gamecenter/43309028,NFL $2K Play-Action (Preseason),2017-08-11 22:49:35.450216
9,700/700,$33,"$20,000",LIVE!,9.0,/contest/gamecenter/43299740,NFL $20K Bootleg (Preseason),2017-08-11 22:49:35.450216


In [192]:
file_name="NFL_contests.csv"
try:
	old_contests=pd.read_csv(file_name)
	full_contests=pd.concat([old_contests,live_contests]).drop_duplicates(how=["link"])
except:
	print("No file %s found. Creating new."%file_name)
	full_contests=live_contests

No file NFL_contests.csv found. Creating new.


In [22]:
rows=[]
for row in table.find("div","grid-canvas").find_all("div","slick-row"):
    row_data={}
    for cell in row.find_all("div"):
        if cell.find("a"):
            a=cell.find("a")
            if a.get("data-tracking"):
                row_data[a.get("data-tracking")]=cell.text
            if cell.text=="Watch":
                row_data['link']=a.get("href")
    rows.append(row_data)
live_contests=pd.DataFrame(rows)#missing some elements that you need to scroll for

In [24]:
live_contests

,Entries,Fee,Payout,Time,name
0,2776/20.5K,$4,"$70,000",06:51:00,MLB $70K Four-Seamer [20 Entry Max]
1,1873/14.7K,$8,"$100,000",06:51:00,MLB $100K Rally Cap
2,2080/9411,$0.25,"$2,000",06:51:00,MLB $2K Quarter Jukebox [Just $0.25!]
3,300/7352,$4,"$25,000",09:51:00,Arcade Mode $25K Four Seamer [20 Entry Max]
4,1452/7058,$1,"$6,000",06:51:00,MLB $6K Solo Shot
5,1383/5882,$0.10,$500,06:51:00,MLB $500 Dime Time [Just $0.10!]
6,774/5882,$0.10,$500,06:51:00,MLB $500 Dime Time [Just $0.10!]
7,838/5165,$55,"$250,000",06:51:00,MLB $250K Full Count [$50K to 1st]
8,388/4705,$0.25,"$1,000",12:51:00,MLB $1K Quarter Jukebox [Just $0.25!] (Night)
9,131/3921,$3,"$10,000",12:51:00,MLB $10K Moonshot [20 Entry Max] (Night)


In [38]:
contests=pd.read_csv("NFL_contests.csv")
contests

,Unnamed: 0,Entries,Fee,Payout,Time,Unnamed: 0.1,link,name,time_pulled
0,0,15.1K/17.6K,$4,"$60,000",LIVE!,0.0,/contest/gamecenter/43299739,NFL $60K Front Four (Preseason),2017-08-11
1,1,7058/7058,$0.25,"$1,500",LIVE!,1.0,/contest/gamecenter/43293788,NFL $1.5K Quarter Jukebox [Just $0.25!] (Prese...,2017-08-11
2,2,4705/4705,$1,"$4,000",LIVE!,2.0,/contest/gamecenter/43293787,NFL $4K First Down (Preseason),2017-08-11
3,3,2370/2823,$0.25,200 Tickets,LIVE!,3.0,/contest/gamecenter/43293832,NFL $2M Play-Action SUPERSatellite [200x] (Pre...,2017-08-11
4,4,1882/1882,$0.25,20 Tickets,LIVE!,4.0,/contest/gamecenter/43293830,NFL $5.55M Fantasy Football Millionaire SUPERS...,2017-08-11
5,5,1176/1176,$3,"$3,000",LIVE!,5.0,/contest/gamecenter/43299737,NFL $3K Play-Action (Preseason),2017-08-11
6,6,976/1058,$1,300 Tickets,LIVE!,6.0,/contest/gamecenter/43293831,NFL $2M Play-Action SUPERSatellite [300x] (Pre...,2017-08-11
7,7,882/882,$1,$750,LIVE!,7.0,/contest/gamecenter/43313515,NFL $750 First Down (Preseason),2017-08-11
8,8,784/784,$3,"$2,000",LIVE!,8.0,/contest/gamecenter/43309028,NFL $2K Play-Action (Preseason),2017-08-11
9,9,700/700,$33,"$20,000",LIVE!,9.0,/contest/gamecenter/43299740,NFL $20K Bootleg (Preseason),2017-08-11


In [40]:
def get_lineups(link):
    sleep(4)
    BASE_URL="https://www.draftkings.com"
    driver.get(BASE_URL+link)
    driver.find_element_by_id("export-lineups-csv").click()
get_lineups("/contest/gamecenter/43299739")

In [41]:
contests['link'].apply(get_lineups)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
Name: link, dtype: object

In [46]:
driver.quit()

In [3]:
import datetime

In [4]:
datetime.datetime.now().strftime("%Y-%m-%d %I")

'2017-08-14 11'